<table>
<tr style="border:none;">
<td style="border:none;"><a href="https://fra.utn.edu.ar/">
<image src="https://raw.githubusercontent.com/frautn/F2-electromagnetismo/main/assets_python/logoUTN-500.svg"  width="100">
</a></td>
<td style="border:none;"><font face="times" size=4>Física 2<br>2024<br>Electromagnetismo<br>Material para trabajo en clase</font>
</td>
</tr>
</table> 

</br>

<table>
<tr style="border:none;">
<td style="border:none;"><a href="https://creativecommons.org/licenses/by-nc-sa/4.0/deed.es"><image src="https://raw.githubusercontent.com/frautn/F2-electromagnetismo/main/docs/assets/img/by-nc-sa.svg" width="70" /></a></td>
<td style="border:none;"><font face="times" size=4>2024 Edgardo Palazzo (epalazzo@fra.utn.edu.ar)</font>
</td>
</tr>
</table>

# Potencial Eléctrico

---

In [1]:
# --------------------------------
#
# En edición.
#
# --------------------------------

**Objetivos**

  - Repasar el cálculo de potencial eléctrico con cargas puntuales.
  - Analizar equipotenciales de diversas configuraciones de cargas puntuales.
  - Obtener el campo eléctrico a partir del potencial eléctrico.

**Tabla de contenidos**<a id='toc0_'></a>    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import sys

np.seterr(divide='ignore')  # Elimina advertencia al dividir por cero.
%precision %.3f

# ------------------------------------
# Lo siguiente es necesario para que funcione en Google Colab.
try:
  from libem import plotE, plotEf, plotEfVector, plotEfvector3d
except:
  # No descarga los archivos (aunque sí descarga .git)
  !git clone -n https://github.com/frautn/F2-electromagnetismo.git --depth 1
  !cd F2-electromagnetismo;git checkout HEAD modulos/libem.py
  sys.path.insert(0, 'F2-electromagnetismo/modulos')
  from libem import plotE, plotEf, plotEfVector, plotEfvector3d

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !pip --quiet install ipympl
  from google.colab import output
  output.enable_custom_widget_manager()
# Fin Google colab.
# ------------------------------------

# Figuras interactivas.
# Comentar si se quiere mantener las figuras luego de cerrar el cuaderno.
# %matplotlib widget


## 1. Potencial eléctrico con cargas puntuales

Si consideramos que el potencial vale cero en el infinito, entonces el potencial eléctrico en una posición $\vec{\text{x}}$ en presencia de una carga puntual $q_0$ ubicada en la posición $\vec{\text{x}}_0$ puede escribirse como:
$$
V(\vec{\text{x}}) = \frac{1}{4\pi\varepsilon_0} \frac{q}{|\vec{\text{r}}|}\, , \tag{1}
$$
donde $|\vec{\text{r}}|$ es la distancia entre la posición de la partícula cargada ($\vec{\text{x}}_0$) y la posición donde se quiere determinar el valor del potencial eléctrico ($\vec{\text{x}}$).  

En presencia de más de una carga eléctrica puntual, el potencial se calcula como la suma de los potenciales de cada carga:
$$
V(\vec{\text{x}}) = \frac{1}{4\pi\varepsilon_0} \sum_{i=1}^N \frac{q_i}{|\vec{\text{r}}_i|}\, , \tag{2}
$$

donde
* $q_i$ es la carga eléctrica de la partícula i-ésima 
* $\vec{\text{r}}_i$ es el vector diferencia entre la posición donde se quiere determinar el valor del potencial eléctrico ($\vec{\text{x}}$) y la posición de la partícula cargada ($\vec{\text{x}}_i$), es decir:
$$
\vec{\text{r}}_i = (x-x_i)\hat{\textbf{i}} + (y-y_i)\hat{\textbf{j}} + (z-z_i)\hat{\textbf{k}} ~ . \tag{3}
$$




In [ ]:
def V1(x,y,z):
    """Calcula las componentes del campo eléctrico en N/C.
    Ingresar valores de x,y,z en metros y q en coulomb.
    """
    k = 9E9   #Constante de Coulomb en las unidades correspondientes.
    
    q = 1E-9
    x0 = 0
    y0 = 0
    z0 = 0

    # El denominador en la ec. 3:
    r32 = ((x - x0)**2 + (y - y0)**2 + (z - z0)**2)**(3/2)

    # Cada una de las componentes de la ec. 3:
    Ei = k * q * (x - x0) / r32
    Ej = k * q * (y - y0) / r32
    Ek = k * q * (z - z0) / r32

    return Ei, Ej, Ek